<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=761d2c55dcbf85ae0bb7d336cf12b9721f0238de6549e926f7cddd2ee465aab9
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-15 14:42:15
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.39 C
-------------------
Today PnL: 93.65 K (0.68%)
Current PnL: -22.16 L (-14.76%)
CY Booked + Current PnL: -10.26 L (-6.83%)
-------------------
Total profit:  1.63 L
Total loss:  -23.79 L
-------------------
Total Booked + Current PnL: 16.72 L (13.58%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.54%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 89.35 L (64.13%)
Deployed:  1.23 C
Current:  1.39 C
CAGR/XIRR %: 7.36%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-20.12,56.0,X-LC,3.71,213501.0,9001.0,14305.0,0.39,4.40,6.70,11.39,10.0,0.63,1.54,13.75,X40N,ATH,PHARMA
76,TTKPRESTIG,770.00,98.29,54.0,M-SC,1.51,85399.0,-15378.0,15474.0,1.01,-15.26,18.12,0.09,245.0,-0.99,0.62,12.20,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,0.74,57.0,X-MC,1.09,211320.0,19578.0,18174.0,1.23,10.21,8.60,19.69,99.0,1.08,1.53,17.70,XY25,NTT,AC
33,ICICIGI,2260.25,-13.84,66.0,X-MC,1.71,148370.0,12237.0,18888.0,8.10,8.99,12.73,22.86,91.0,0.65,1.07,22.74,X40,ATH,INSURANCE
56,RELIANCE,1533.00,-14.41,47.0,X-LC,2.97,214484.0,4118.0,24666.0,-0.07,1.96,11.50,13.68,37.0,0.17,1.55,18.80,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,ICICIGI,2260.25,-13.84,66.0,X-MC,1.71,148370.0,12237.0,18888.0,8.10,8.99,12.73,22.86,91.0,0.65,1.07,22.74,X40,ATH,INSURANCE
49,NESTLEIND,1377.00,-6.95,62.0,X-LC,2.40,286205.0,20779.0,36005.0,4.05,7.83,12.58,21.40,11.0,0.58,2.07,14.99,XY25,NTT,FMCG
81,VALIANTORG,838.00,-283.41,31.0,H-SC,3.52,101915.0,-61690.0,183009.0,3.04,-37.71,179.57,74.15,139.0,-0.34,0.74,30.64,XR,NTT,CHEMICALS
6,ASIANTILES,137.00,6555.56,50.0,L-SC,6.05,73023.0,-20787.0,97405.0,2.80,-22.16,133.39,81.67,269.0,-0.21,0.53,41.41,XR,NTT,CERAMICS
5,ASIANPAINT,3465.66,-17.11,46.0,X-LC,5.36,209264.0,-42504.0,95717.0,2.58,-16.88,45.74,21.13,27.0,-0.44,1.51,12.56,X40,ATH,PAINTS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
10,BANDHANBNK,400.0,-12.63,47.0,H-SC,2.95,217871.0,-60688.0,318135.0,-3.50,-21.79,146.02,92.42,151.0,-0.19,1.58,25.91,XY24,NTT,BANKS
15,CAMPUS,393.0,-23.81,55.0,M-SC,2.70,158666.0,-9109.0,64958.0,-1.86,-5.43,40.94,33.28,210.0,-0.14,1.15,22.98,XY24,NTT,FOOTWEAR
37,INDIGOPNTS,1408.0,93.64,29.0,M-SC,3.36,127038.0,-47521.0,47550.0,-1.74,-27.22,37.43,0.02,221.0,-1.00,0.92,10.50,OX40N,NTT,PAINTS
72,TATAMOTORS,1065.0,-55.15,12.0,X-LC,3.36,141243.0,-135280.0,245353.0,-1.61,-48.92,173.71,39.81,54.0,-0.55,1.02,0.00,XY24,NTT,AUTO
34,ICICIPRULI,790.0,-22.72,41.0,X-MC,2.09,133001.0,-2787.0,45540.0,-1.60,-2.05,34.24,31.48,107.0,-0.06,0.96,9.82,X40,ATH,INSURANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-186.36,68.0,M-SC,16.22,148293.0,-927.0,145105.0,1.45,-0.62,97.85,96.62,208.0,-0.01,1.07,60.49,XY25,NTT,FINANCE
59,SAIL,228.00,41.94,47.0,M-MC,12.28,224463.0,-499.0,168145.0,1.23,-0.22,74.91,74.53,192.0,-0.00,1.62,31.96,XY24,BTT,STEEL
36,INDIAMART,4810.62,-55.57,37.0,H-SC,8.27,123946.0,610.0,131023.0,0.16,0.49,105.71,106.72,119.0,0.00,0.90,23.99,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,107.70,60.0,H-SC,11.55,130977.0,-11502.0,31028.0,1.95,-8.07,23.69,13.70,163.0,-0.37,0.95,54.59,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,98.29,54.0,M-SC,1.51,85399.0,-15378.0,15474.0,1.01,-15.26,18.12,0.09,245.0,-0.99,0.62,12.20,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.79,52.0,H-SC,1.87,222606.0,-47061.0,83388.0,0.77,-17.45,37.46,13.47,138.0,-0.56,1.61,13.19,XY24,NTT,PAINTS
17,CERA,9475.0,-23.63,41.0,H-SC,2.11,139322.0,-36581.0,78605.0,-0.55,-20.80,56.42,23.89,149.0,-0.47,1.01,20.32,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-24.72,43.0,H-MC,7.02,101821.0,-30014.0,72242.0,0.45,-22.77,70.95,32.03,98.0,-0.42,0.74,14.05,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,47.87,52.0,M-LC,6.30,207130.0,3960.0,38174.0,1.70,1.95,18.43,20.74,55.0,0.10,1.50,7.60,XY25,NTT,FINANCE
31,HINDZINC,730.22,37.47,70.0,M-LC,8.95,224242.0,19166.0,93397.0,1.95,9.35,41.65,54.89,52.0,0.21,1.62,35.69,X5K,ATH,METALS
85,WIPRO,420.00,-11.52,56.0,M-LC,5.76,155174.0,4229.0,105223.0,0.75,2.80,67.81,72.51,53.0,0.04,1.12,8.99,XR,NTT,IT
78,UNIONBANK,163.00,-11.63,56.0,M-LC,9.52,158498.0,17658.0,26834.0,1.75,12.54,16.93,31.59,66.0,0.66,1.15,42.35,XY24,NTT,BANKS
11,BANKINDIA,190.00,-24.22,63.0,H-MC,12.74,194557.0,14749.0,97668.0,1.77,8.20,50.20,62.52,88.0,0.15,1.41,45.09,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,37.47,70.0,M-LC,8.95,224242.0,19166.0,93397.0,1.95,9.35,41.65,54.89,52.0,0.21,1.62,35.69,X5K,ATH,METALS
11,BANKINDIA,190.00,-24.22,63.0,H-MC,12.74,194557.0,14749.0,97668.0,1.77,8.20,50.20,62.52,88.0,0.15,1.41,45.09,XR,NTT,BANKS
85,WIPRO,420.00,-11.52,56.0,M-LC,5.76,155174.0,4229.0,105223.0,0.75,2.80,67.81,72.51,53.0,0.04,1.12,8.99,XR,NTT,IT
36,INDIAMART,4810.62,-55.57,37.0,H-SC,8.27,123946.0,610.0,131023.0,0.16,0.49,105.71,106.72,119.0,0.00,0.90,23.99,AR,ATH,MISC
84,WHIRLPOOL,2270.00,-47.73,38.0,M-SC,3.97,88432.0,-3066.0,81817.0,2.17,-3.35,92.52,86.07,223.0,-0.04,0.64,28.42,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-55.15,12.0,X-LC,3.36,141243.0,-135280.0,245353.0,-1.61,-48.92,173.71,39.81,54.0,-0.55,1.02,0.00,XY24,NTT,AUTO
51,QUESS,986.00,-48.21,30.0,X-SC,38.41,51399.0,-13607.0,164533.0,0.23,-20.93,320.11,232.18,198.0,-0.08,0.37,0.23,XY24,NTT,MISC
12,BATAINDIA,2096.00,-37.58,34.0,X-SC,4.42,93392.0,-35278.0,80579.0,0.09,-27.42,86.28,35.20,219.0,-0.44,0.68,7.19,X40,NTT,FOOTWEAR
55,RELAXO,1176.00,-41.43,34.0,X-SC,3.76,79872.0,-65288.0,141214.0,-0.50,-44.98,176.80,52.31,136.0,-0.46,0.58,6.91,X40N,NTT,FOOTWEAR
28,HAVELLS,2069.16,-7.74,35.0,X-MC,1.50,228540.0,-19191.0,94250.0,1.10,-7.75,41.24,30.30,92.0,-0.20,1.65,4.52,X40,ATH,ELECTRICAL


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-2.40,46.0,H-LC,0.93,156750.0,-29345.0,76823.0,0.89,-15.77,49.01,25.51,15.0,-0.38,1.13,15.51,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,0.74,57.0,X-MC,1.09,211320.0,19578.0,18174.0,1.23,10.21,8.60,19.69,99.0,1.08,1.53,17.70,XY25,NTT,AC
28,HAVELLS,2069.16,-7.74,35.0,X-MC,1.50,228540.0,-19191.0,94250.0,1.10,-7.75,41.24,30.30,92.0,-0.20,1.65,4.52,X40,ATH,ELECTRICAL
33,ICICIGI,2260.25,-13.84,66.0,X-MC,1.71,148370.0,12237.0,18888.0,8.10,8.99,12.73,22.86,91.0,0.65,1.07,22.74,X40,ATH,INSURANCE
49,NESTLEIND,1377.00,-6.95,62.0,X-LC,2.40,286205.0,20779.0,36005.0,4.05,7.83,12.58,21.40,11.0,0.58,2.07,14.99,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-55.15,12.0,X-LC,3.36,141243.0,-135280.0,245353.0,-1.61,-48.92,173.71,39.81,54.0,-0.55,1.02,0.00,XY24,NTT,AUTO
51,QUESS,986.00,-48.21,30.0,X-SC,38.41,51399.0,-13607.0,164533.0,0.23,-20.93,320.11,232.18,198.0,-0.08,0.37,0.23,XY24,NTT,MISC
82,VBL,671.64,-21.40,40.0,X-LC,10.67,283336.0,-32506.0,145181.0,0.57,-10.29,51.24,35.67,5.0,-0.22,2.05,2.14,X40N,ATH,FMCG
9,BAJAJHFL,181.50,-15.75,36.0,X-MC,5.66,177563.0,-22943.0,120104.0,0.97,-11.44,67.64,48.45,90.0,-0.19,1.28,2.39,X40N,ATH,FINANCE
73,TCS,4389.97,-30.29,43.0,X-LC,12.24,277234.0,-68722.0,135429.0,-1.56,-19.86,48.85,19.28,1.0,-0.51,2.01,2.49,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.21,30.0,X-SC,38.41,51399.0,-13607.0,164533.0,0.23,-20.93,320.11,232.18,198.0,-0.08,0.37,0.23,XY24,NTT,MISC
55,RELAXO,1176.00,-41.43,34.0,X-SC,3.76,79872.0,-65288.0,141214.0,-0.50,-44.98,176.80,52.31,136.0,-0.46,0.58,6.91,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-37.58,34.0,X-SC,4.42,93392.0,-35278.0,80579.0,0.09,-27.42,86.28,35.20,219.0,-0.44,0.68,7.19,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-5.38,49.0,X-SC,1.16,107663.0,5659.0,38263.0,1.72,5.55,35.54,43.06,122.0,0.15,0.78,23.61,X40N,ATH,MISC
32,HONAUT,58357.33,-23.12,45.0,X-SC,2.31,108420.0,-19518.0,66657.0,0.14,-15.26,61.48,36.84,143.0,-0.29,0.78,11.00,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-30.29,43.0,X-LC,12.24,277234.0,-68722.0,135429.0,-1.56,-19.86,48.85,19.28,1.0,-0.51,2.01,2.49,X40,ATH,IT
39,INFY,2275.00,-19.24,47.0,X-LC,7.64,314068.0,1180.0,170508.0,-1.03,0.38,54.29,54.87,3.0,0.01,2.27,7.05,X40,BTT,IT
41,ITC,452.00,-39.61,43.0,X-LC,2.83,195429.0,-4709.0,25601.0,0.72,-2.35,13.10,10.44,4.0,-0.18,1.41,3.08,X40,NTT,FMCG
82,VBL,671.64,-21.40,40.0,X-LC,10.67,283336.0,-32506.0,145181.0,0.57,-10.29,51.24,35.67,5.0,-0.22,2.05,2.14,X40N,ATH,FMCG
1,ABB,7934.00,-40.55,47.0,H-LC,7.42,248328.0,-13291.0,132508.0,1.05,-5.08,53.36,45.57,7.0,-0.10,1.80,5.67,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,80.00,-43.66,52.0,L-SC,28.82,82579.0,-30970.0,71018.0,1.80,-27.27,86.00,35.27,268.0,-0.44,0.60,102.11,XR,NTT,HOTELS
48,MASFIN,398.61,-20.96,39.0,H-SC,9.86,89595.0,-8385.0,29987.0,-0.32,-8.56,33.47,22.05,152.0,-0.28,0.65,30.23,XR,ATH,FINANCE
77,UJJIVANSFB,60.00,107.70,60.0,H-SC,11.55,130977.0,-11502.0,31028.0,1.95,-8.07,23.69,13.70,163.0,-0.37,0.95,54.59,OX40N,NTT,BANKS
68,SURYODAY,240.00,57.89,54.0,H-SC,9.93,144871.0,-34200.0,99932.0,-0.37,-19.10,68.98,36.71,135.0,-0.34,1.05,43.74,XR,NTT,BANKS
33,ICICIGI,2260.25,-13.84,66.0,X-MC,1.71,148370.0,12237.0,18888.0,8.10,8.99,12.73,22.86,91.0,0.65,1.07,22.74,X40,ATH,INSURANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-186.36,68.0,M-SC,16.22,148293.0,-927.0,145105.0,1.45,-0.62,97.85,96.62,208.0,-0.01,1.07,60.49,XY25,NTT,FINANCE
31,HINDZINC,730.22,37.47,70.0,M-LC,8.95,224242.0,19166.0,93397.0,1.95,9.35,41.65,54.89,52.0,0.21,1.62,35.69,X5K,ATH,METALS
68,SURYODAY,240.00,57.89,54.0,H-SC,9.93,144871.0,-34200.0,99932.0,-0.37,-19.10,68.98,36.71,135.0,-0.34,1.05,43.74,XR,NTT,BANKS
4,ANGELONE,3033.00,13.27,66.0,X-SC,9.88,200662.0,9656.0,48038.0,1.77,5.06,23.94,30.21,157.0,0.20,1.45,26.99,X40N,NTT,FINANCE
67,STARHEALTH,761.00,16.85,62.0,H-SC,8.29,260032.0,-21091.0,153965.0,2.26,-7.50,59.21,47.26,144.0,-0.14,1.88,39.32,XY24,NTT,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.53
1,25,44.02
2,50,75.88


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.37
LC    33.72
MC    21.91
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.62
X40      14.81
XY25     12.19
XR       11.02
X40N     10.11
AR        8.20
OX40N     7.67
X200      1.82
X5K       1.62
SR        1.12
MH        0.82
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.59
X-LC    22.55
X-MC    16.14
M-SC    12.67
M-LC     5.39
H-LC     4.75
X-SC     4.64
H-MC     3.84
M-MC     1.62
L-SC     1.47
L-LC     1.03
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.11,-5.19,39.86
IT,12.98,-17.16,80.53
FINANCE,12.20,-12.04,62.82
MISC,6.91,-20.49,87.25
BANKS,6.45,-12.26,71.32
PAINTS,5.59,-19.48,37.30
ELECTRICAL,5.36,-10.96,49.89
INSURANCE,3.91,-2.15,40.34
AUTO,2.81,-47.63,118.20


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3342172.0,22
XR,1388720.0,14
AR,1195512.0,9
X40,789700.0,10
X40N,627919.0,8
OX40N,588998.0,10
XY25,497773.0,8
SR,266487.0,2
X5K,93397.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3464446.0,25
M-SC,1291292.0,15
X-LC,1120018.0,13
X-MC,1093897.0,11
X-SC,539284.0,6
H-MC,349967.0,3
H-LC,282999.0,3
M-LC,263628.0,4
L-SC,258552.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1204425.0      6
           XR         825583.0      7
           AR         822288.0      5
M-SC       XY24       699479.0      6
X-MC       XY24       589015.0      4
X-LC       X40        496325.0      5
           XY24       309684.0      2
X-SC       X40N       294172.0      4
H-SC       OX40N      275226.0      4
           SR         266487.0      2
X-LC       X40N       213643.0      3
X-MC       X40        212796.0      4
H-LC       AR         209331.0      2
H-MC       XY24       180057.0      1
X-MC       XY25       171982.0      2
L-SC       XR         168423.0      2
M-MC       XY24       168145.0      1
X-SC       XY24       164533.0      1
M-SC       AR         163893.0      2
           OX40N      151401.0      4
           XY25       145105.0      1
           XR         131414.0      2
X-MC       X40N       120104.0      1
M-LC       XR         105223.0      1
X-LC       XY25       100366.0      3
H-MC       XR          97668.0      1
M-LC       X5K         93397.0      1
L-SC       OX40N       90129.0      1
X-SC       X40         80579.0      1
H-LC       X200        73668.0      1
H-MC       OX40N       72242.0      1
H-SC       MH          70437.0      1
L-MC       XR          60409.0      1
L-LC       XY25        42146.0      1
M-LC       XY25        38174.0      1
           XY24        26834.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
